# Smart Facility: Safety Basic Dataset - Demo Notebook

Demo 1: Filtering/querying the dataset based on metadata.  
Demo 2: Visualizing labels for a specific video.

Note: this notebook relies on the `infinity-tools` python module, which can be installed by following the [installation instructions](https://github.com/toinfinityai/infinity-tools#installation).

In [1]:
import os
import glob
import json
import shutil
import pandas as pd
import zipfile
from pycocotools.coco import COCO
from IPython.display import clear_output
from infinity_tools.visionfit import vis
from infinity_tools.common.vis.videos import parse_video_frames, stack_videos

## Demo 1: Dataset Filtering

We first show how to load the dataset's metadata into an easily filtered dataframe.

In [2]:
PATH_TO_DATASET_FOLDER = "PATH_TO_DATASET_FOLDER"

In [3]:
all_param_paths = sorted(glob.glob(os.path.join(PATH_TO_DATASET_FOLDER, "*_params.json")))

metadata = []
for param_path in all_param_paths:
    metadata_dict = {}
    metadata_dict["video_number"] = os.path.basename(param_path).split("_")[0]
    parameters = json.load(open(param_path))["params"]
    metadata_dict.update(parameters)
    metadata.append(metadata_dict)

metadata = pd.DataFrame(metadata)
metadata.head()

,video_number,video_duration,frame_rate,scene,max_dimension,num_people,random_seed
0,00000,10.0,12,WAREHOUSE_0003,1080,5,53659
1,00001,10.0,12,WAREHOUSE_0003,1080,5,67874
2,00002,10.0,12,WAREHOUSE_0002,1080,5,4195
3,00003,10.0,12,WAREHOUSE_0002,1080,5,73521
4,00004,10.0,12,WAREHOUSE_0001,1080,5,22241


You can now filter and/or query the dataset for specific properties, as shown in the example below.

In [4]:
filtered_samples = metadata.query("scene == 'WAREHOUSE_0001'")
display(filtered_samples.head())

,video_number,video_duration,frame_rate,scene,max_dimension,num_people,random_seed
4,00004,10.0,12,WAREHOUSE_0001,1080,5,22241
6,00006,10.0,12,WAREHOUSE_0001,1080,5,61404
7,00007,10.0,12,WAREHOUSE_0001,1080,5,63206
8,00008,10.0,12,WAREHOUSE_0001,1080,5,65632
9,00009,10.0,12,WAREHOUSE_0001,1080,5,68861


## Demo 2: Label Visualization

We next show how to visualize labels for a specific inventory video. We do this using the dataset sample provided in the repo.

In [5]:
def visualize_all_labels(video_rgb_path: str) -> str:
    """Visualizes RGB, bounding boxes and segmentation masks in single video."""

    output_directory = os.path.dirname(video_rgb_path)
    labels_path = video_rgb_path.replace(".mp4", "_labels.json")
    seg_zip_path = video_rgb_path.replace(".mp4", "_segmentation.zip")
    
    seg_extracted_path = seg_zip_path.replace(".zip", "")
    os.makedirs(seg_extracted_path, exist_ok=True)
    vis.unzip(seg_zip_path, seg_extracted_path)

    fps = 12
    imgs = parse_video_frames(video_rgb_path)
    coco = COCO(labels_path)

    bounding_box_path = vis.create_bounding_boxes_video(
        os.path.join(output_directory, "labels_bounding_box.mp4"), 
        imgs, 
        fps, 
        coco,
        add_text=True,
    )
    
    segmentation_path = vis.create_segmentation_video(
        os.path.join(output_directory, "labels_segmentation.mp4"),
        seg_extracted_path,
        fps,
        imgs.shape[2],
        imgs.shape[1]
    )
    
    skeleton_path = vis.create_keypoint_connections_video(
        os.path.join(output_directory, "labels_skeleton.mp4"), 
        imgs, 
        fps, 
        coco
    )

    cuboid_path = vis.create_cuboids_video(
        os.path.join(output_directory, "labels_cuboid.mp4"), 
        imgs, 
        fps, 
        coco
    )
        
    shutil.rmtree(seg_extracted_path)
    clear_output()
    
    print(f"The resulting label videos can be viewed at:")
    for e in [bounding_box_path, segmentation_path, skeleton_path, cuboid_path]:
        print(f"\t{e}")

In [6]:
video_rgb_path = "./sample_data/00004.mp4"
visualize_all_labels(video_rgb_path)

The resulting label videos can be viewed at:
	./sample_data/labels_bounding_box.mp4
	./sample_data/labels_segmentation.mp4
	./sample_data/labels_skeleton.mp4
	./sample_data/labels_cuboid.mp4
